In [1]:
1+2


3

Haha,got it work

# Why don't we start the project NAS?

## First we need to find out how do the first paper build the Architecture and train datasets

In [ ]:
#---------------------------------- Get familiar with the code in autokeras-----------------------------#
# ----------------classifier.py
# init->fit->evaluate

#fit->
#    # Transform x,y_train.
#    # Create the searcher and save on disk  
#    # Divide training data into training and testing data
#    # LOOP
       $$$run_searcher_once(train_data, test_data, self.path)$$$
         if len(self.load_searcher().history) >= Constant.MAX_MODEL_NUM#1000:
                break
#    #LOOP END
#
#
#         # 
#         ###run_searcher_once()->autokeras.utils.pickle_from_file() CALL searcher.search()
#             input_shape = x_train.shape[1:]
            n_classes = self.y_encoder.n_classes #An integer, the number of classes
            self.searcher_args['n_classes'] = n_classes #An integer, the number of classes
            self.searcher_args['input_shape'] = input_shape #A tuple. e.g. (28, 28, 1)
            self.searcher_args['path'] = self.path#The path to the directory to save the searcher
            self.searcher_args['verbose'] = self.verbose#A boolean. Whether to output the intermediate information to stdout.
            $$$$  searcher = BayesianSearcher(**self.searcher_args) $$$$
            self.save_searcher(searcher)
            self.searcher = True
#-----------------Search.py

#           ->__init__()
                 default_model_len=Constant.MODEL_LEN,#3
                 default_model_width=Constant.MODEL_WIDTH,#64
                 beta=Constant.BETA,#0.576
                 kernel_lambda=Constant.KERNEL_LAMBDA#1.0
                if trainer_args is None:
                    trainer_args = {}
                if 'max_iter_num' not in self.trainer_args:
                self.trainer_args['max_iter_num'] = Constant.SEARCH_MAX_ITER#200
                self.gpr = IncrementalGaussianProcess(kernel_lambda)
                self.search_tree = SearchTree()
                self.training_queue = []
                self.x_queue = []
                self.y_queue = []
                self.beta = beta
                if t_min is None:
                    t_min = Constant.T_MIN#0.0001
                self.t_min = t_min
#           ->search()
#             ->init_search()  start
#------------------generator.py
                graph = DefaultClassifierGenerator(self.n_classes,
                self.input_shape).generate(self.default_model_len,
                                          self.default_model_width)
                    $$$$$generate$$$$$
                        model_width=Constant.MODEL_WIDTH#64):
                        pooling_len = int(model_len#3 / 4)
                        graph = Graph(self.input_shape, False)
#------------------Graph.py                                          
                          $$$$$$Graph.__init__start$$$$$$
                                    self.weighted = weighted#A boolean of whether the weights and biases in the neural network should be included in the graph.
                                    self.node_list = []#A list of integers, the indices of the list are the identifiers.
                                    self.layer_list = []#A list of stub layers, the indices of the list are the identifiers.
                                    # node id start with 0
                                    self.node_to_id = {}#A dict instance mapping from node integers to their identifiers.
                                    self.layer_to_id = {}# A dict instance mapping from stub layers to their identifiers.
                                    self.layer_id_to_input_node_ids = {}#A dict instance mapping from layer identifiers to their input nodes identifiers.
                                    self.layer_id_to_output_node_ids = {}
                                    self.adj_list = {}#A two dimensional list. The adjacency list of the graph. The first dimension is identified by tensor identifiers. In each edge list, the elements are two-element tuples of (tensor identifier, layer identifier).
                                    self.reverse_adj_list = {}#A reverse adjacent list in the same format as adj_list.
                                    self.operation_history = []#A list saving all the network morphism operations.

                                    self.vis = None#A dictionary of temporary storage for whether an local operation has been done during the network morphism.
                                    self._add_node(Node(input_shape))
                          $$$$$$Graph.__init__end$$$$$$
                        temp_input_channel = self.input_shape[-1]
                        output_node_id = 0
                        for i in range(model_len):
                            output_node_id = graph.add_layer(StubReLU(), output_node_id)
                            output_node_id = graph.add_layer(StubConv(temp_input_channel, model_width, kernel_size=3), output_node_id)
                            output_node_id = graph.add_layer(StubBatchNormalization(model_width), output_node_id)
                            output_node_id = graph.add_layer(StubDropout(Constant.CONV_DROPOUT_RATE), output_node_id)
                            temp_input_channel = model_width
                            if pooling_len == 0 or ((i + 1) % pooling_len == 0 and i != model_len - 1):
                                output_node_id = graph.add_layer(StubPooling(), output_node_id)

                        output_node_id = graph.add_layer(StubFlatten(), output_node_id)
                        output_node_id = graph.add_layer(StubDense(graph.node_list[output_node_id].shape[0], self.n_classes),
                                                         output_node_id)
                        graph.add_layer(StubSoftmax(), output_node_id)
                        return graph
                model_id = self.model_count
                self.model_count += 1
                self.training_queue.append((graph, -1, model_id))
                self.descriptors.append(graph.extract_descriptor())
                for child_graph in default_transform(graph):
                    child_id = self.model_count
                    self.model_count += 1
                    self.training_queue.append((child_graph, model_id, child_id))
                    self.descriptors.append(child_graph.extract_descriptor())
                if self.verbose:
                    print('Initialization finished.') 
#            ->init_search() end
#            # Start the new process for training.
            graph, father_id, model_id = self.training_queue.pop(0)
            if self.verbose:
                print('Training model ', model_id)
                multiprocessing.set_start_method('spawn', force=True)
                pool = multiprocessing.Pool(1)
                $$$$$$$train_results = pool.map_async(train, [(graph, train_data, test_data, self.trainer_args,
                                                        os.path.join(self.path, str(model_id) + '.png'), self.verbose)])                        
#            # Do the search in current thread.
            if not self.training_queue:
                new_graph, new_father_id = self.maximize_acq()
                new_model_id = self.model_count
                self.model_count += 1
                self.training_queue.append((new_graph, new_father_id, new_model_id))
                descriptor = new_graph.extract_descriptor()
                self.descriptors.append(new_graph.extract_descriptor())

            accuracy, loss, graph = train_results.get()[0]
            pool.terminate()
            pool.join()
            self.add_model(accuracy, loss, graph, model_id)
            self.search_tree.add_child(father_id, model_id)
            self.gpr.fit(self.x_queue, self.y_queue)
            self.x_queue = []
            self.y_queue = []

            pickle_to_file(self, os.path.join(self.path, 'searcher'))
            self.export_json(os.path.join(self.path, 'history.json'))


# ->DefaultClassifierGenerator->Graph


### Minst.py

In [ ]:
### Minst dataset using autokeras

In [ ]:
from keras.datasets import mnist
from autokeras.classifier import ImageClassifier

if __name__ == '__main__':
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))

    clf = ImageClassifier(verbose=True, augment=False)
    clf.fit(x_train, y_train, time_limit=0.1 * 60 * 60)
    clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)
    y = clf.evaluate(x_test, y_test)
    print(y * 100)

Using TensorFlow backend.


Initializing search.
Plotting model init  1111111111
Initialization finished.
before the new proc
Plotting model  0
Training model  0


### How to visualization

#### code alike

In [10]:
# check keras status
import keras
from keras.utils.vis_utils import plot_model
# check new plot func 
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot



#    plot_model(model,
               to_file='model.png',
               show_shapes=False,
               show_layer_names=True,
               rankdir='TB')
#plot(model, to_file='model.png')                    
    """Converts a Keras model to dot format and save to a file.

    # Arguments
        model: A Keras model instance
        to_file: File name of the plot image.
        show_shapes: whether to display shape information.
        show_layer_names: whether to display layer names.
        rankdir: `rankdir` argument passed to PyDot,
            a string specifying the format of the plot:
            'TB' creates a vertical plot;
            'LR' creates a horizontal plot.
    """